In [1]:
#Declare and import libraries
import pandas as pd
import numpy as np
#Read data from local JSON into initial dataframe HoPdf
HoPdf = pd.read_json('purchase_data.json',encoding='UTF-8')
HoPdf.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [2]:
#Using value_counts on the SN series, count the total number of players (unique SN's) and store it in a variable
total_players_series = HoPdf['SN'].value_counts()
total_players_series
total_players_int = len(total_players_series)
total_players_int

573

In [27]:
#Use groupby on primary DF then count to identify unique items in a series
unique_weapons_groupbydf = HoPdf.groupby(['Item Name'])
unique_weapon_series = unique_weapons_groupbydf['Price'].count()
print(unique_weapon_series)
unique_weapon_int = len(unique_weapon_series)
print("The number of unique items purchased was: " +str(unique_weapon_int))

Item Name
Abyssal Shard                                3
Aetherius, Boon of the Blessed               4
Agatha                                       5
Alpha                                        7
Alpha, Oath of Zeal                          7
Alpha, Reach of Ending Hope                  6
Amnesia                                      6
Apocalyptic Battlescythe                     4
Arcane Gem                                  11
Avenger                                      6
Azurewrath                                   5
Betrayal, Whisper of Grieving Widows        11
Betrayer                                     3
Blade of the Grave                           7
Blazefury, Protector of Delusions            2
Blazeguard, Reach of Eternity                6
Blindscythe                                  5
Blood Infused Guardian                       5
Blood-Forged Skeletal Spine                  2
Bloodlord's Fetish                           1
Bone Crushing Silver Skewer                  3
Bon

In [3]:
#Using the mean method on the Price series, store this value in a variable
mean_item_price = HoPdf['Price'].mean()
mean_item_price = round(mean_item_price,2)
print("The average item price is: $"+str(mean_item_price))

The average item price is: $2.93


In [4]:
#Because the JSON is a record of in-game transactions over some period, the number of rows 
#in any category is the number of total transactions
total_purchases = HoPdf['Price'].count()
total_purchases

780

In [5]:
#Total revenue can be used by extracting the Price series from the HoPdf then use the sum method
price_series = HoPdf['Price']
price_series.sum()

2286.33

In [19]:
#Create a new data frame from the original dataframe with just screennames and genders
unique_sn_gender = HoPdf[['SN','Gender']]
#Drop duplicated SN values, but keep the first instance
unique_sn_gender = unique_sn_gender.drop_duplicates(subset=['SN'],keep='first')
unique_sn_gender_valcounts = unique_sn_gender['Gender'].value_counts() #dataframe
unique_sn_gender_valcounts

Male                     465
Female                   100
Other / Non-Disclosed      8
Name: Gender, dtype: int64

In [12]:
#Use the total_players_int to convert counts to a rounded percentage and rename the column to show it is a percentage value
percent_gender = pd.DataFrame(round((unique_sn_gender['Gender'].value_counts() / total_players_int)*100))
percent_gender = percent_gender.rename(columns={'Gender':'Gender (%)'})
percent_gender.head()

,Gender (%)
Male,81.0
Female,17.0
Other / Non-Disclosed,1.0


In [31]:
#Create a new data frame using groupby on Gender and count price
purchase_count_gender = HoPdf.groupby(['Gender'])['Price'].count()
purchase_count_gender = pd.DataFrame(purchase_count_gender)
purchase_count_gender = purchase_count_gender.join(unique_sn_gender_valcounts,how='right')
purchase_count_gender = purchase_count_gender.join(round(HoPdf.groupby(['Gender'])['Price'].mean(),2),rsuffix='mean')
purchase_count_gender = purchase_count_gender.join(round(HoPdf.groupby(['Gender'])['Price'].sum(),2),rsuffix='sum')
purchase_count_gender['Normalized Totals'] = round(purchase_count_gender['Pricesum']/purchase_count_gender['Gender'],2)
purchase_count_gender = purchase_count_gender.rename(columns={'Price':'Count of Purchases by Gender','Pricemean':'Mean Price($)','Pricesum':'Total Revenue($)','Gender':'Players','Normalized Totals':'Normalized $/Player'})
purchase_count_gender

,Count of Purchases by Gender,Players,Mean Price($),Total Revenue($),Normalized $/Player
Male,633,465,2.95,1867.68,4.02
Female,136,100,2.82,382.91,3.83
Other / Non-Disclosed,11,8,3.25,35.74,4.47


In [39]:
#Create a list of age bins up to 51 since the max age (HoPdf['Age'].max() returns 45) is 45
age_bins = [0,9,14,19,24,29,34,39,44,49,54]
#Make bin labels that can be used to easily parse the data when tabulated into a df
bin_labels = ['0-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54']
#perform the cut on the original dataframe on the ['Age'] series
HoPdf['Age Group'] = pd.cut(HoPdf['Age'],bins=age_bins,labels=bin_labels)
#Use groupby to isolate data by the age bins
age_group_groupby = HoPdf.groupby(['Age Group'])
#As above, use the count, mean, and sum methods on relevant data series to establish
age_group_purchase_counts = age_group_groupby['SN'].count()
age_group_purchase_mean = round(age_group_groupby['Price'].mean(),2)
age_group_purchase_sum = round(age_group_groupby['Price'].sum(),2)
age_group_purchase_relfreq = round(age_group_purchase_sum/age_group_purchase_counts,2)
#BEGIN PASTA FUCK MY LIFE!!!
unique_sn_age = HoPdf[['SN','Age Group']]
#Drop duplicated SN values, but keep the first instance
unique_sn_age = unique_sn_age.drop_duplicates(subset=['SN'],keep='first')
#END PASTA
age_demographics_df = pd.DataFrame(age_group_purchase_counts)
age_demographics_df = age_demographics_df.rename(columns={'SN':'Number of Purchases'})
age_demographics_df['Average Purchase Price ($)'] = age_group_purchase_mean
age_demographics_df['Total Revenue ($)'] = age_group_purchase_sum
age_demographics_df['Normalized Totals ($/purchase)'] = age_group_purchase_relfreq
age_demographics_df["Unique SN's"] = unique_sn_age
age_demographics_df

ValueError: Wrong number of items passed 2, placement implies 1

In [24]:
value_counts_snHoPdf['SN'].value_counts() #series

SyntaxError: invalid syntax (<ipython-input-24-f1d95827b9b2>, line 1)

,Age,Gender,Item ID,Item Name,Price,SN,Age Group
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [49]:
#unique_sn_age = HoPdf[['Age Group']['SN']]
HoPdf[['Age Group']['SN']]

TypeError: list indices must be integers or slices, not str